# Business Problem

### For this project, a client has request the production of a MySQL database on Movies from a subset of IMDB's publicly available dataset. They would like an analysis of this database to help determine what makes a movie successful and recommendations on how to make a successful movie.

# Import Libraries

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [5]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"

aka_url="https://datasets.imdbws.com/title.akas.tsv.gz"

rating_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

In [9]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [10]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [11]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
2,tt0000003,6.5,1840
3,tt0000004,5.5,178
4,tt0000005,6.2,2622


# Clean Data

## Basics

- Replace "\N" with np.nan
- Eliminate movies that are null for runtimeMinutes
- Eliminate movies that are null for genre
- keep only titleType==Movie
- keep startYear 2000-2022
- Eliminate movies that include "Documentary" in genre (see tip below)
- Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)


### Replace "\N" with np.nan

In [12]:
basics.replace("\\N", np.nan, inplace=True)

In [13]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


### Eliminate movies that are null for runtimeMinutes

In [14]:
basics.dropna(subset=['runtimeMinutes'], inplace=True)
basics['runtimeMinutes'].isna().sum()

0

### Eliminate movies that are null for genre

In [15]:
basics.dropna(subset=['genres'], inplace=True)
basics['genres'].isna().sum()

0

### keep only titleType==Movie and display results

In [16]:
basics['titleType'].unique()

array(['short', 'movie', 'tvShort', 'tvMovie', 'tvSeries', 'tvEpisode',
       'tvMiniSeries', 'video', 'tvSpecial', 'videoGame'], dtype=object)

In [17]:
is_movie = basics['titleType'] == 'movie'
basics = basics[is_movie]
basics['titleType'].unique()

array(['movie'], dtype=object)

In [18]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,NaN,100,"Documentary,News,Sport"
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"


### keep startYear 2000-2022

In [19]:
basics['startYear'].info()

<class 'pandas.core.series.Series'>
Int64Index: 384760 entries, 8 to 9981235
Series name: startYear
Non-Null Count   Dtype 
--------------   ----- 
378243 non-null  object
dtypes: object(1)
memory usage: 5.9+ MB


In [20]:
basics['startYear'] = basics['startYear'].astype('float')

In [21]:
years = (basics['startYear'] >= 2000) & (basics['startYear'] <= 2021)
basics = basics[years]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
13082,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021.0,NaN,94,Documentary
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
42384,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013.0,NaN,120,"Drama,History"
61115,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67667,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama


### Eliminate movies that include "Documentary" in genre

In [22]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

basics[is_documentary]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


# AKAs

### Keep Only US Movies

In [23]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [24]:
us = akas['region'] == 'US'

In [25]:
akas = akas[us]
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,\N,imdbDisplay,\N,0
14,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,\N,imdbDisplay,\N,0
36,tt0000005,1,Blacksmithing Scene,US,\N,alternative,\N,0
41,tt0000005,6,Blacksmith Scene #1,US,\N,alternative,\N,0


### Replace "\N" with np.nan

In [26]:
akas.replace("\\N", np.nan, inplace=True)
akas[akas.eq('\\N').any(1)]

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle


### keep only US movies for 'Basics'
- filter after 'AKAs'.

In [27]:
keepers = basics['tconst'].isin(akas['titleId'])
keepers

34803       True
42384       True
61115       True
67667       True
86799       True
           ...  
9980958     True
9980967     True
9981006    False
9981051     True
9981135    False
Name: tconst, Length: 138734, dtype: bool

In [28]:
basics = basics[keepers]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
42384,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013.0,NaN,120,"Drama,History"
61115,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67667,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86799,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


# Ratings

### Replace "\N" with np.nan (if any)

In [29]:
ratings[ratings.eq('\\N').any(1)]

,tconst,averageRating,numVotes


### Keep Only US Movies

In [30]:
ratings_us = ratings['tconst'].isin(akas['titleId'])

In [31]:
ratings = ratings[ratings_us]
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
4,tt0000005,6.2,2622
5,tt0000006,5.1,182
6,tt0000007,5.4,820


# Data Summary

In [32]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81843 entries, 34803 to 9981051
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          81843 non-null  object 
 1   titleType       81843 non-null  object 
 2   primaryTitle    81843 non-null  object 
 3   originalTitle   81843 non-null  object 
 4   isAdult         81843 non-null  object 
 5   startYear       81843 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  81843 non-null  object 
 8   genres          81843 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.2+ MB


In [33]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1449308 entries, 5 to 36450698
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1449308 non-null  object
 1   ordering         1449308 non-null  int64 
 2   title            1449308 non-null  object
 3   region           1449308 non-null  object
 4   language         3981 non-null     object
 5   types            980948 non-null   object
 6   attributes       46927 non-null    object
 7   isOriginalTitle  1447966 non-null  object
dtypes: int64(1), object(7)
memory usage: 99.5+ MB


In [34]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 502824 entries, 0 to 1326934
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         502824 non-null  object 
 1   averageRating  502824 non-null  float64
 2   numVotes       502824 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.3+ MB


# Create Data Folder

In [35]:
# example making new folder with os
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

['tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'title_basics.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 '.ipynb_checkpoints',
 'title_akas.csv.gz',
 'title_ratings.csv.gz']

### Saves files to Data Folder

In [36]:
basics.to_csv('Data/title_basics.csv.gz', compression='gzip', index=False)

In [37]:
akas.to_csv('Data/title_akas.csv.gz', compression='gzip', index=False)

In [38]:
ratings.to_csv('Data/title_ratings.csv.gz', compression='gzip', index=False)

# DF to CSV's read in from Data

In [6]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [7]:
akas = pd.read_csv(aka_url, sep='\t', low_memory=False)

In [8]:
ratings = pd.read_csv(rating_url, sep='\t', low_memory=False)